In [1]:
import os
import time
import dotenv
import pprint
from IPython import display
from langchain_neo4j import GraphCypherQAChain, Neo4jGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama.chat_models import ChatOllama

dotenv.load_dotenv(".env")

True

In [ ]:
graph = Neo4jGraph(
    url=os.environ["DATABASE_HOST"],
    username=os.environ["DATABASE_USERNAME"],
    password=os.environ["DATABASE_PASSWORD"],
    database=os.environ["DATABASE_LARGE"],
    enhanced_schema=True
)

In [3]:
graph.refresh_schema()
print(graph.schema)

Node properties:
- **Regulation**
  - `title`: STRING Example: "Peraturan Menteri Komunikasi dan Informatika Nomor"
  - `id`: INTEGER Min: 199901036100000, Max: 202403006100000
  - `type`: STRING Available options: ['PERMENKOMINFO', 'PP', 'UU']
  - `number`: INTEGER Min: 1, Max: 80
  - `year`: INTEGER Min: 1999, Max: 2024
  - `institution`: STRING Available options: ['Kementerian Komunikasi dan Informatika', 'Pemerintah Pusat']
  - `issue_place`: STRING Available options: ['Jakarta']
  - `issue_date`: DATE Min: 1999-09-08, Max: 2024-09-30
  - `effective_date`: DATE Min: 1999-09-23, Max: 2024-10-03
  - `subjects`: LIST Min Size: 1, Max Size: 4
  - `is_amendment`: BOOLEAN 
  - `reference_url`: STRING Example: "https://peraturan.bpk.go.id/Details/159911/permenk"
  - `download_url`: STRING Example: "https://peraturan.bpk.go.id/Download/152473/Permen"
  - `download_name`: STRING Example: "PERMENKOMINFO_2007_011"
- **Definition**
  - `name`: STRING Example: "Telekomunikasi"
  - `id`: INTEGER

In [4]:
from langchain_core.prompts.prompt import PromptTemplate

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

Following are some examples that you can use as a reference to create Cypher code according to user questions.

User query   : Apa isi pasal 100 peraturan pemerintah / PP nomor 90 tahun 2020?
Cypher query : MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'PP' AND r.number = 90 AND r.year = 2020 AND a.number = '100'
RETURN a.text AS text

User query   : Apa isi pasal selanjutnya dari pasal 100 undang-undang / UU nomor 90 tahun 2020?
Cypher query : MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'UU' AND r.number = 90 AND r.year = 2020 AND a.number = '100'
RETURN next_article.text AS text

Don't use `real_text` attribute! But use `text` attribute instead.

Now, make a cypher code for the following user questions.

The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

In [ ]:
chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0, api_key=os.environ["GOOGLE_API_KEY"]),
    # qa_llm=ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0.1),
    qa_llm=ChatOllama(model="gemma3:4b-it-q4_K_M", temperature=0.1),
    # cypher_llm=ChatOllama(model="qwen2.5-coder:7b-instruct-q4_K_M", temperature=0.1),
    graph=graph,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    # return_direct=True,
    # return_intermediate_steps=True,
    # use_function_response=True,  # Jika pakai ini maka qa_prompt tidak akan digunakan
    # function_response_system="response like a football commentator!",  # Hanya jika pakai use_function_response=True
    # exclude_types=["Ineffective"],  # Misalnya aja mau exclude sesuatu, tapi kayanya tidak
    validate_cypher=True,
    allow_dangerous_requests=True
)

# chain.graph_schema
# Bandingkan pakai few shot dan tanpa few shot
# Perlihatkan kekuatan knowledge graph, contoh:
# 1. Pasal paling banyak direfer oleh pasal lain
# 2. Clustering pasal
# Pakai yg API dan local, bandingkan mana yang terbaik

In [20]:
start_time = time.time()
response = chain.invoke(
    "Apa isi lengkap dari pasal 28 UU Nomor 11 Tahun 2008?"
)

pprint.pprint(response)
print(f"Execution time: {time.time() - start_time} seconds")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'
RETURN a.text AS text

Full Context:
[{'text': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 28:\n(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.\n(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).'}]

> Finished chain.
{'query': 'Apa isi lengkap dari pasal 28 UU Nomor 11 Tahun 2008?',
 'result': 'Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita '
           'bohong dan menyesatkan yang mengakibatkan kerugian konsume

In [21]:
start_time = time.time()
response = chain.invoke(
    "Apa nomor pasal setelah pasal 1 UU Nomor 19 Tahun 2016? Dan apa isi lengkapnya?"
)

pprint.pprint(response)
print(f"Execution time: {time.time() - start_time} seconds")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)-[:NEXT_ARTICLE]->(next_article)
WHERE r.type = 'UU' AND r.number = 19 AND r.year = 2016 AND a.number = '1'
RETURN next_article.number AS number, next_article.text AS text

Full Context:
[{'number': '2', 'text': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB I - KETENTUAN UMUM, Pasal 2:\nUndang-Undang ini berlaku untuk setiap Orang yang melakukan perbuatan hukum sebagaimana diatur dalam Undang-Undang ini, baik yang berada di wilayah hukum Indonesia maupun di luar wilayah hukum Indonesia, yang memiliki akibat hukum di wilayah hukum Indonesia dan/atau di luar wilayah hukum Indonesia dan merugikan kepentingan Indonesia.'}]

> Finished chain.
{'query': 'Apa nomor pasal setelah pasal 1 UU Nomor 19 Tahun 2016? Dan apa isi '
          'lengkapnya?',
 'result': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan '
           'T

In [11]:
start_time = time.time()
response = chain.invoke(
    "Berapa jumlah pasal yang masih berlaku (effective) di UU Nomor 11 Tahun 2008?\n"
    "Berapa yang tidak berlaku?"
    "Berapa total pasal di sana?\n"
    "Dan berapa persentase yang masih berlaku dan tidak berlaku tersebut?"
)

# pprint.pprint(response)
display.display(display.Markdown(response["result"]))
print(f"Execution time: {time.time() - start_time} seconds")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Regulation {type: 'UU', number: 11, year: 2008})
WITH r
MATCH (r)-[:HAS_ARTICLE]->(a:Article)
WITH r, COUNT(a) AS totalArticles
OPTIONAL MATCH (r)-[:HAS_ARTICLE]->(e:Effective)
WITH r, totalArticles, COUNT(e) AS effectiveArticles
OPTIONAL MATCH (r)-[:HAS_ARTICLE]->(i:Ineffective)
WITH r, totalArticles, effectiveArticles, COUNT(i) AS ineffectiveArticles
RETURN 
    effectiveArticles,
    ineffectiveArticles,
    totalArticles,
    (toFloat(effectiveArticles) / totalArticles) * 100 AS effectivePercentage,
    (toFloat(ineffectiveArticles) / totalArticles) * 100 AS ineffectivePercentage

Full Context:
[{'effectiveArticles': 41, 'ineffectiveArticles': 13, 'totalArticles': 54, 'effectivePercentage': 75.92592592592592, 'ineffectivePercentage': 24.074074074074073}]

> Finished chain.


41 pasal dari UU Nomor 11 Tahun 2008 masih berlaku, dan 13 pasal tidak berlaku. Total pasal adalah 54, dengan 41 pasal masih berlaku (75.93%) dan 13 pasal tidak berlaku (24.07%).

Execution time: 11.277246713638306 seconds


In [12]:
start_time = time.time()
response = chain.invoke(
    "Apa saja nomor pasal yang sudah tidak berlaku (effective) di UU Nomor 11 Tahun 2008?"
)

# pprint.pprint(response)
display.display(display.Markdown(response["result"]))
print(f"Execution time: {time.time() - start_time} seconds")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(i:Ineffective)
WHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008
RETURN i.number AS NomorPasalTidakBerlaku

Full Context:
[{'NomorPasalTidakBerlaku': '1'}, {'NomorPasalTidakBerlaku': '5'}, {'NomorPasalTidakBerlaku': '13'}, {'NomorPasalTidakBerlaku': '17'}, {'NomorPasalTidakBerlaku': '26'}, {'NomorPasalTidakBerlaku': '27'}, {'NomorPasalTidakBerlaku': '28'}, {'NomorPasalTidakBerlaku': '29'}, {'NomorPasalTidakBerlaku': '31'}, {'NomorPasalTidakBerlaku': '36'}]

> Finished chain.


1, 5, 13, 17, 26, 27, 28, 29, 31, 36.

Execution time: 9.218703508377075 seconds


In [13]:
start_time = time.time()
response = chain.invoke(
    "Jelaskan apa saja perbedaan isi pasal 28 UU nomor 11 tahun 2008 sebelum dan sesudah diamandemen?"
)

# pprint.pprint(response)
display.display(display.Markdown(response["result"]))
print(f"Execution time: {time.time() - start_time} seconds")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)
WHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '28'
OPTIONAL MATCH (a)-[:AMENDED_BY]->(amended_article)
RETURN a.text AS OriginalArticle, amended_article.text AS AmendedArticle

Full Context:
[{'OriginalArticle': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB VII - PERBUATAN YANG DILARANG, Pasal 28:\n(1) Setiap Orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik.\n(2) Setiap Orang dengan sengaja dan tanpa hak menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan individu dan/atau kelompok masyarakat tertentu berdasarkan atas suku, agama, ras, dan antargolongan (SARA).', 'AmendedArticle': 'Undang-undang (UU) Nomor 1 Tahun 2024 tentang Perubahan Kedua atas Undang-Undang Nomor 11 Tahun 200

Sebelum diamandemen, Pasal 28 UU Nomor 11 Tahun 2008 menyatakan bahwa setiap orang dengan sengaja dan tanpa hak menyebarkan berita bohong dan menyesatkan yang mengakibatkan kerugian konsumen dalam Transaksi Elektronik, serta menyebarkan informasi yang ditujukan untuk menimbulkan rasa kebencian atau permusuhan terhadap kelompok tertentu berdasarkan SARA. Setelah diamandemen, pasal tersebut menambahkan bahwa setiap orang dengan sengaja menyebarkan Informasi Elektronik dan/atau Dokumen Elektronik yang diketahui memuat pemberitahuan bohong yang menimbulkan kerusuhan di masyarakat.

Execution time: 25.70857548713684 seconds


In [ ]:
start_time = time.time()
response = chain.invoke(
    "Apa saja nomor pasal yang sudah tidak berlaku (effective) di UU Nomor 11 Tahun 2008?\n"
    "Lalu apa saja isi pasal amandemennya dan apa bedanya dengan isi pasal sebelum diamandemen?\n"
    "Berikan 2 contoh saja.\n"
    "Lalu jelaskan apa yang berbeda sebelum dan sesudah amandemen"
)

# pprint.pprint(response)
display.display(display.Markdown(response["result"]))
print(f"Execution time: {time.time() - start_time} seconds")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (r:Regulation)-[:HAS_ARTICLE]->(i:Ineffective)
WHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008
WITH r, i
MATCH (i)-[:AMENDED_BY]->(a:Article)
RETURN i.number AS IneffectiveArticleNumber,
       i.text AS IneffectiveArticleText,
       a.text AS AmendedArticleText
LIMIT 2

Full Context:
[{'IneffectiveArticleNumber': '1', 'IneffectiveArticleText': 'Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, BAB I - KETENTUAN UMUM, Pasal 1:\nDalam Undang-Undang ini yang dimaksud dengan:\n(1) Informasi Elektronik adalah satu atau sekumpulan data elektronik, termasuk tetapi tidak terbatas pada tulisan, suara, gambar, peta, rancangan, foto, electronic data interchange (EDI), surat elektronik (electronic mail), telegram, teleks, telecopy atau sejenisnya, huruf, tanda, angka, Kode Akses, simbol, atau perforasi yang telah diolah yang memiliki arti atau dapat dipahami oleh orang yang mampu 

Oke, mari kita analisis perubahan dalam UU Nomor 11 Tahun 2008 setelah adanya amandemen (UU Nomor 1 Tahun 2024):

**Nomor Pasal yang Tidak Berlaku (Effective):**

*   Pasal 5 UU Nomor 11 Tahun 2008 (sebelum amandemen)

**Isi Pasal Sebelum dan Sesudah Amandemen (dengan 2 Contoh):**

**Pasal 5 (Sebelum Amandemen - UU 11 Tahun 2008):**

*   (1) Informasi Elektronik dan/atau Dokumen Elektronik dan/atau hasil cetaknya merupakan alat bukti hukum yang sah.
*   (2) Informasi Elektronik dan/atau Dokumen Elektronik dan/atau hasil cetaknya sebagaimana dimaksud pada ayat (1) merupakan perluasan dari alat bukti yang sah sesuai dengan Hukum Acara yang berlaku di Indonesia.
*   (3) Informasi Elektronik dan/atau Dokumen Elektronik dinyatakan sah apabila menggunakan Sistem Elektronik sesuai dengan ketentuan yang diatur dalam Undang-Undang ini.
*   (4) Ketentuan mengenai Informasi Elektronik dan/atau Dokumen Elektronik sebagaimana dimaksud pada ayat (1) tidak berlaku untuk:
    *   a. surat yang menurut Undang-Undang harus dibuat dalam bentuk tertulis; dan
    *   b. surat beserta dokumennya yang menurut Undang-Undang harus dibuat dalam bentuk akta notaril atau akta yang dibuat oleh pejabat pembuat akta.

**Pasal 5 (Sesudah Amandemen - UU 1 Tahun 2024):**

*   (1) Informasi Elektronik dan/atau Dokumen Elektronik dan/atau hasil cetaknya merupakan alat bukti hukum yang sah.
*   (2) Informasi Elektronik dan/atau Dokumen Elektronik dan/atau hasil cetaknya sebagaimana dimaksud pada ayat (1) merupakan perluasan dari alat bukti yang sah sesuai dengan Hukum Acara yang berlaku di Indonesia.
*   (3) Informasi Elektronik dan/atau Dokumen Elektronik dinyatakan sah apabila menggunakan Sistem Elektronik sesuai dengan ketentuan yang diatur dalam Undang-Undang ini.
*   (4) Ketentuan mengenai Informasi Elektronik dan/atau Dokumen Elektronik sebagaimana dimaksud pada ayat (1) tidak berlaku dalam hal diatur lain dalam Undang-Undang.

**Perbedaan Utama (2 Contoh):**

1.  **Pengecualian Penerapan:**
    *   **Sebelum Amandemen:** Pasal 5 (sebelum) secara spesifik menyatakan bahwa ketentuan mengenai Informasi Elektronik dan Dokumen Elektronik *tidak berlaku* untuk surat yang harus dibuat dalam bentuk tertulis dan surat yang harus dibuat dalam bentuk akta notaril. Ini memberikan batasan yang jelas.
    *   **Sesudah Amandemen:** Pasal 5 (sesudah) menyatakan bahwa ketentuan *tidak berlaku* dalam hal diatur lain dalam Undang-Undang. Ini menjadi lebih umum dan membuka kemungkinan adanya peraturan lain yang dapat mengatur penggunaan bukti elektronik dalam kasus-kasus tertentu.

2.  **Ruang Lingkup Penerapan:**
    *   **Sebelum Amandemen:**  Fokus pada batasan-batasan penggunaan bukti elektronik, terutama dalam konteks surat-surat tertentu.
    *   **Sesudah Amandemen:** Lebih fleksibel, karena menyatakan bahwa ketentuan tidak berlaku jika ada peraturan lain yang lebih spesifik. Ini berarti penggunaan bukti elektronik dapat diatur lebih lanjut melalui undang-undang atau peraturan lain.

Semoga penjelasan ini membantu!

Execution time: 131.9371018409729 seconds
